# Stochastisches Bilevel-Modell für PV-Investitions- und Betriebsentscheidungen

Dieses Notebook implementiert ein stochastisches Bilevel-Modell, das eine Investitionsentscheidung für eine Photovoltaik-(PV)-Anlage und dazugehörige operative Entscheidungen (Netzbezug und Einspeisung) integriert. Der Code nutzt Julia, JuMP und Ipopt und bildet das in der zugehörigen PDF beschriebene Modell ab.

In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `c:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject`
┌ Warning: The project dependencies or compat requirements have changed since the manifest was last resolved.
│ It is recommended to `Pkg.resolve()` or consider `Pkg.update()` if necessary.
└ @ Pkg.API C:\Users\Reintjes\.julia\juliaup\julia-1.11.2+0.x64.w64.mingw32\share\julia\stdlib\v1.11\Pkg\src\API.jl:1200


In [2]:
Pkg.add("MathOptInterface")
Pkg.add("JuMP")
Pkg.add("Distributions")
Pkg.add("HiGHS")
Pkg.add("Clustering")
Pkg.add("Distances")
Pkg.add("Plots")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("Ipopt")
Pkg.add("BilevelJuMP")
Pkg.add("Juniper") #new

    Updating registry at `C:\Users\Reintjes\.julia\registries\General.toml`
   Resolving package versions...
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Project.toml`
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Project.toml`
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Project.toml`
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Project.toml`
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Manifest.to

In [3]:
using JuMP, Distributions, Clustering, Distances, Plots, Statistics, StatsBase, Ipopt, Juniper

## Parameterdefinitionen

Hier werden die grundlegenden Parameter definiert, wie etwa die Anzahl der Zeitschritte, Investitionskosten, Strompreise sowie Parameter zur Modellierung der PV-Leistung und Emissionen.

In [4]:
# Anzahl der Zeitperioden (z.B. 12 Monate * 24 Stunden)
T = 288

# Anzahl der Szenarien pro Zeitschritt
num_scenarios = 5

# Investitionskosten (Beispielwert)
#C_inv = 10 * 12

# Social Cost of Carbon (€/t CO2)
SCC = 100

# Kosten für Netzbezug und Erlös aus Einspeisung (€/kWh)
c_grid_buy = 0.47
c_grid_feed = 0.083

# Emissionsintensität (tCO2/kWh)
e_grid = 0.00038

# PV-Parameter
η_0 = 0.18      # Nominale PV-Wirkungsgrad
β = 0.004       # Temperaturkoeffizient pro °C
T_ref = 25      # Referenztemperatur in °C

# -- Parameter für Investitionskosten-Formel nach Bergner (2019)
I0   = 1.923    # €/kW
p    = 0.16     # Skalierungsparameter
MwSt = 0.19     # 19% Mehrwertsteuer
r    = 0.0275   # Zinssatz pro Jahr (2,75%)
n    = 25       # Nutzungsdauer in Jahren

# -- Aus der Annuitätenformel: a = r * (1+r)^n / ((1+r)^n - 1)
#    Diesen Faktor brauchen wir, um die einmaligen Investitionskosten
#    auf jährliche Kosten A umzurechnen.
function annuity_factor(r, n)
    return r * (1+r)^n / ((1+r)^n - 1)
end
a = annuity_factor(r, n)

0.055839973499395165

## Definition der Effizienzfunktion

Die Funktion `efficiency` berechnet den temperaturabhängigen PV-Wirkungsgrad.

In [5]:
function efficiency(T_t)
    return η_0 * (1 - β * (T_t - T_ref))
end

efficiency (generic function with 1 method)

## Erzeugung der stochastischen Verteilungen

In diesem Abschnitt wird für jeden Zeitschritt ein Vektor von Verteilungen erstellt, der Solarstrahlung, Temperatur und Verbrauch abbildet. Dabei werden saisonale und tageszeitliche Effekte berücksichtigt.

In [6]:
using Distributions, Random

# Funktion zur Erzeugung eines Verteilungsvektors für einen Zeitschritt
function generate_distributions(si::Float64, consumption_mean::Float64, temp_mean::Float64, hour::Int)
    if hour < 6 || hour > 22
        # Nachtstunden: nahezu keine Sonneneinstrahlung
        solar = Beta(1, 1000)  
        temperature = Normal(temp_mean - 4, 9)
        consumption = Normal(consumption_mean/(consumption_mean - 1), consumption_mean / 11)
    else
        # Tagesstunden: solare Einstrahlung mit Beta-Verteilung
        β_param = ((si * (1 + si) / (si / 10)) - 1) * (1 - si)
        α_param = si * β_param / (1 - si)
        solar = Beta(α_param, β_param)
        temperature = Normal(temp_mean, 9)
        consumption = Normal(consumption_mean, consumption_mean / 11)
    end
    return [solar, consumption, temperature]
end

# Initialisierung des Verteilungsvektors für jeden Zeitschritt
dist_vec = Vector{Vector{Distribution}}(undef, 12 * 24)

for month in 1:12
    for hour in 1:24
        # Saisonale und tageszeitabhängige Parameter
        si = 0.25 + 0.1 * sin(2π * month / 12)  
        si = hour < 6 || hour > 22 ? 0.0 : si  
        consumption_mean = 90.0 + 10 * sin(2π * hour / 24)
        temp_mean = 13.0 + 5 * cos(2π * month / 12)
        
        # Speichern des Verteilungsvektors
        dist_vec[(month - 1) * 24 + hour] = generate_distributions(si, consumption_mean, temp_mean, hour)
    end
end

## Erzeugung der Dummy-Szenarien

Für jeden Zeitschritt werden 5 Szenarien für Solarstrahlung, Verbrauch und Temperatur erzeugt. Außerdem werden die Szenario-Wahrscheinlichkeiten (gleichverteilt) festgelegt.

In [7]:
# Gesamtzahl der Zeitschritte
T_total = 12 * 24

# Erzeugen der Dummy-Szenarien für Solar (Index 1 im Verteilungsvektor)
S_t_omega = [ [rand(dist_vec[t][1]) for ω in 1:num_scenarios] for t in 1:T_total ]

# Erzeugen der Dummy-Szenarien für Verbrauch (Index 2 im Verteilungsvektor)
L_t_omega = [ [rand(dist_vec[t][2]) for ω in 1:num_scenarios] for t in 1:T_total ]

# Erzeugen der Dummy-Szenarien für Temperatur (Index 3 im Verteilungsvektor)
T_t_omega = [ [rand(dist_vec[t][3]) for ω in 1:num_scenarios] for t in 1:T_total ]

# Szenario-Wahrscheinlichkeiten (uniform verteilt)
Φ_t_omega = [ [1/num_scenarios for ω in 1:num_scenarios] for t in 1:T_total ]

288-element Vector{Vector{Float64}}:
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 ⋮
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]
 [0.2, 0.2, 0.2, 0.2, 0.2]

## Testausgabe für den ersten Zeitschritt

Wir geben die für den ersten Zeitschritt (erste Stunde im Januar) generierten Werte aus.

In [8]:
println("Erste Stunde im Januar:")
println("Solar: ", S_t_omega[1])
println("Verbrauch: ", L_t_omega[1])
println("Temperatur: ", T_t_omega[1])
println("Szenario-Wahrscheinlichkeiten: ", Φ_t_omega[1])

Erste Stunde im Januar:
Solar: [0.0006041084494892507, 0.0010541562166015768, 0.0066311315184416795, 0.00011371794046073516, 0.0003081510212139357]
Verbrauch: [0.18371635445886603, 5.824140975682989, 5.03092715148676, -9.565328812707097, -5.769054228035606]
Temperatur: [6.272086806266157, 15.600603907534838, 18.17846550051094, 8.763638107363413, 14.580206950550508]
Szenario-Wahrscheinlichkeiten: [0.2, 0.2, 0.2, 0.2, 0.2]


## Erstellung und Lösung des Optimierungsmodells

Das Optimierungsmodell wird in zwei Ebenen aufgeteilt:

- **Obere Ebene:** Investitionsentscheidung (Variable y_PV) unter Berücksichtigung der jährlichen Investitionskosten.
- **Untere Ebene:** Operative Entscheidungen (Netzbezug G_in und Einspeisung G_out) für jeden Zeitschritt und jedes Szenario. Die KKT-Bedingungen der unteren Ebene werden verwendet, um das bilevel-Problem in ein Single-Level-Problem zu transformieren.

Anhand der Energiebilanz wird sichergestellt, dass der Verbrauch durch PV-Erzeugung, Netzbezug und Einspeisung gedeckt wird.

In [16]:
model = Model(Ipopt.Optimizer)

# Obere Ebene: Investitionsentscheidung (Leader)
@variable(model, 0 <= y_PV <= 30)

# Untere Ebene: Operationelle Variablen (Follower)
@variable(model, G_in[1:T, 1:num_scenarios] >= 0)   # Netzimport
@variable(model, G_out[1:T, 1:num_scenarios] >= 0)   # Netzexport

# Dualvariablen für KKT-Bedingungen
@variable(model, λ[1:T, 1:num_scenarios])
@variable(model, nu_in[1:T, 1:num_scenarios] >= 0)
@variable(model, nu_out[1:T, 1:num_scenarios] >= 0)

# ----------------------------
# NICHTLINEARE AUSDRÜCKE FÜR INVESTITIONSKOSTEN
# ----------------------------
# I_pv = (I0 * y_PV^(-p)) * y_PV * (1 + MwSt)
#      = I0 * y_PV^(1-p) * (1 + MwSt)
# A = I_pv * a  (jährliche Investitionskosten)
@NLexpression(model, I_pv, I0 * (0.2*y_PV)^(1 - p) * (1 + MwSt))
@NLexpression(model, A, I_pv * a)

for t in 1:T, ω in 1:num_scenarios
    # Berechnung der PV-Erzeugung als fester Wert unter Nutzung der Effizienzfunktion
    P_PV_val = efficiency(T_t_omega[t][ω]) * S_t_omega[t][ω]
    
    # Primal-Nebenbedingung: Energiebilanz
    @constraint(model, L_t_omega[t][ω] == y_PV * P_PV_val + G_in[t, ω] - G_out[t, ω])
    
    # Stationaritätsbedingungen (Gradienten der Lagrangefunktion)
    @constraint(model, c_grid_buy - λ[t, ω] - nu_in[t, ω] == 0)
    @constraint(model, -c_grid_feed + λ[t, ω] - nu_out[t, ω] == 0)
    
    # Komplementaritätsbedingungen (als nichtlineare Nebenbedingungen)
    @NLconstraint(model, nu_in[t, ω] * G_in[t, ω] == 0)
    @NLconstraint(model, nu_out[t, ω] * G_out[t, ω] == 0)
end

# Zielfunktion: Minimierung der Investitionskosten plus der gewichteten Betriebs- und Emissionskosten
@NLobjective(model, Min,
(A/365)*12 +
    sum(Φ_t_omega[t][ω] * (c_grid_buy * G_in[t, ω] - c_grid_feed * G_out[t, ω] + SCC * e_grid * G_in[t, ω])
        for t in 1:T, ω in 1:num_scenarios)
)

optimize!(model)

This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:    15840
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     8641

Total number of variables............................:     7201
                     variables with only lower bounds:     5760
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:     7200
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.2240215e+00 1.29e+02 5.99e-09  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

## Ausgabe der optimalen PV-Kapazität

Nach der Optimierung wird die optimale Investitionsentscheidung (PV-Kapazität) ausgegeben.

In [17]:
println("Optimale PV-Kapazität: ", value(y_PV))

Optimale PV-Kapazität: 29.999997763285837


## Big M

In [18]:
using JuMP, Juniper, Ipopt, HiGHS
ipopt = optimizer_with_attributes(Ipopt.Optimizer, "print_level" => 0)
highs = optimizer_with_attributes(HiGHS.Optimizer, "output_flag" => false)
model = Model(
    optimizer_with_attributes(
        Juniper.Optimizer,
        "nl_solver" => ipopt,
        "mip_solver" => highs,
    ),
)

# Obere Ebene: Investitionsentscheidung (Leader)
@variable(model, 0 <= y_PV <= 30)

# Untere Ebene: Operationelle Variablen (Follower)
@variable(model, G_in[1:T, 1:num_scenarios] >= 0)   # Netzimport
@variable(model, G_out[1:T, 1:num_scenarios] >= 0)   # Netzexport

# Dualvariablen für KKT-Bedingungen
@variable(model, λ[1:T, 1:num_scenarios])
@variable(model, nu_in[1:T, 1:num_scenarios] >= 0)
@variable(model, nu_out[1:T, 1:num_scenarios] >= 0)
@variable(model, z_in[1:T, 1:num_scenarios], Bin)
@variable(model, z_out[1:T, 1:num_scenarios], Bin)

# I_pv = (I0 * y_PV^(-p)) * y_PV * (1 + MwSt)
#      = I0 * y_PV^(1-p) * (1 + MwSt)
@NLexpression(model, I_pv, I0 * (0.2*y_PV)^(1 - p) * (1 + MwSt))

# Jährliche Investitionskosten via Annuitätenformel
@NLexpression(model, A, I_pv * a)


M = 1_000  # Big-M Wert

for t in 1:T, ω in 1:num_scenarios
    # Berechnung der PV-Erzeugung als fester Wert unter Nutzung der Effizienzfunktion
    P_PV_val = efficiency(T_t_omega[t][ω]) * S_t_omega[t][ω]
    
    # Primal-Nebenbedingung: Energiebilanz
    @constraint(model, L_t_omega[t][ω] == y_PV * P_PV_val + G_in[t, ω] - G_out[t, ω])
    
    # Stationaritätsbedingungen (Gradienten der Lagrangefunktion)
    @constraint(model, c_grid_buy - λ[t, ω] - nu_in[t, ω] == 0)     #TODO: Chatgpt hat einfach ehrenlosen fehler gemacht hier
    @constraint(model, -c_grid_feed + λ[t, ω] - nu_out[t, ω] == 0)
    
    # Big-M Formulierung für Komplementaritätsbedingungen
    @constraint(model, G_in[t, ω] <= M * z_in[t, ω])
    @constraint(model, nu_in[t, ω] <= M * (1 - z_in[t, ω]))
    @constraint(model, G_out[t, ω] <= M * z_out[t, ω])
    @constraint(model, nu_out[t, ω] <= M * (1 - z_out[t, ω]))
end

# Zielfunktion: Minimierung der Investitionskosten plus der gewichteten Betriebs- und Emissionskosten
@NLobjective(model, Min,
    (A/365)*12 +
    sum(Φ_t_omega[t][ω] * (c_grid_buy * G_in[t, ω] - c_grid_feed * G_out[t, ω] + SCC * e_grid * G_in[t, ω])
        for t in 1:T, ω in 1:num_scenarios)
)

optimize!(model)
println("Optimale PV-Kapazität: ", value(y_PV))

nl_solver   : MathOptInterface.OptimizerWithAttributes(Ipopt.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("print_level") => 0])
mip_solver  : MathOptInterface.OptimizerWithAttributes(HiGHS.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("output_flag") => false])
log_levels  : [:Options, :Table, :Info]

#Variables: 10081
#IntBinVar: 2880
Obj Sense: Min

Start values are not feasible.
Status of relaxation: LOCALLY_SOLVED
Time for relaxation: 2.367000102996826
Relaxation Obj: 9126.311408940957

       MIPobj              NLPobj       Time 
     1415.1574             3.0e-6        4.4 

FP: 5.325999975204468 s
FP: 1 round
FP: Obj: 9126.29335829878
Obj: 9126.29335829878
Optimale PV-Kapazität: 29.999997763297465
